# Machine Translation Project (English to Spanish)

In [1]:
import pathlib
import random
import tensorflow as tf
import string
import tensorflow.strings as tf_strings
import tensorflow.data as tf_data
import re
from keras.layers import TextVectorization
import keras

from keras import layers
import json

### Verify access to the GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8156203116117668526
xla_global_id: -1
]


### Download and prepare the data
source :"http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

In [3]:
text_file =  keras.utils.get_file(
    fname = "spa-eng.zip",
    origin = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract = True,
)

text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

with open(text_file, "r") as f:
    lines = f.read().split("\n")[:-1]
    
text_pairs = []

for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [4]:
random.shuffle(text_pairs)

In [5]:
for i in range(5):
    print(text_pairs[i])

('Please explain how to get there.', '[start] Por favor explique cómo llegar allí. [end]')
('Not everything in this world is measured in money.', '[start] No todo en este mundo se mide en dinero. [end]')
("Tom knows some of Mary's friends.", '[start] Tom conoce a algunos de los amigos de Mary. [end]')
('I gave Tom a chance.', '[start] Le di a Tom una oportunidad. [end]')
('We are glad to have you in our class.', '[start] Nos alegramos de teneros en nuestra clase. [end]')


Structure of the Dataset

In [6]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [7]:
# parameters
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

## Vectorize the data

In [8]:
def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# vectorization
eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

spa_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

#save the vectorization layers
eng_vectorization_config = eng_vectorization.get_config()
eng_vectorization_config.pop('standardize', None)
eng_vocab = eng_vectorization.get_vocabulary()
with open('eng_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vectorization_config, f)
    
with open('eng_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    
spa_vectorization_config = spa_vectorization.get_config()
spa_vectorization_config.pop('standardize', None)
spa_vocab = spa_vectorization.get_vocabulary()
with open('spa_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vectorization_config, f)
    
with open('spa_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vocab, f)
    

def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )
    
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
    

In [9]:
for inputs,targets in train_ds.take(1):
    print(inputs["encoder_inputs"].shape)
    print(targets.shape)

(64, 20)
(64, 20)


### Model Architecture
![Encoder-Decoder](images/encoder-decoder-context.png)
![Encoder-Decoder](images/encoder-decoder-translation.png)
![Attention Mechanism](images/attention.png)

In [10]:
# Creating an Encoder
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
        else:
            padding_mask = None
            
        attention_output = self.attention(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config
    
# Creating a Positional Embedding
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        if mask is not None:
            return tf.not_equal(inputs, 0)
        else:
            return None
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "sequence_length": self.sequence_length,
            "embed_dim": self.embed_dim,
        })
        return config
    
# Creating a Decoder
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        else:
            padding_mask = None
            
        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )
        
        out_2 = self.layernorm_2(out_1 + attention_output_2)
        proj_output = self.dense_proj(out_2)
        
        return self.layernorm_3(out_2 + proj_output)
    
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, tf.int32)
        mask = tf.reshape(mask,(1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.convert_to_tensor([1, 1]),
            ],
            axis = 0,
        )
        return tf.tile(mask, mult)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


In [11]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 256         
latent_dim = 2048
num_heads =8

#Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)



In [12]:
epochs = 10

transformer.summary()

transformer.compile(
    "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)


Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [1]:
transformer.save("transformer_model")

NameError: name 'transformer' is not defined

In [17]:
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ensure you have these variables defined properly
# Initialize `eng_tokenizer`
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer()
# Fit tokenizer on your training data or load a pre-fitted tokenizer
# eng_tokenizer.fit_on_texts(train_texts) or load pre-trained tokenizer

# Initialize `spa_vectorization` (it should be a similar tokenizer or vectorization method for Spanish)
spa_vectorization = tf.keras.preprocessing.text.Tokenizer()
# Fit tokenizer on your training data or load a pre-fitted tokenizer
# spa_vectorization.fit_on_texts(train_texts) or load pre-trained tokenizer

# Define the sequence length (ensure it matches with your model's expected input size)
sequence_length = 20  # example length, adjust as necessary

# Define test pairs and strip_chars (examples provided)
test_pairs = [("hello", "hola"), ("goodbye", "adios")]  # replace with your actual test pairs
strip_chars = '.,!?'  # characters to remove from input sentences

# Get vocabulary and create index lookup
spa_vocab = spa_vectorization.word_index
spa_index_lookup = dict(zip(spa_vocab.values(), spa_vocab.keys()))
max_decoded_sentence_length = sequence_length
eng_vectorization_config = {'max_len': sequence_length}

def eng_vectorization(sentences):
    sequences = eng_tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=eng_vectorization_config['max_len'], padding='post')

def decode_sentence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization.texts_to_sequences([decoded_sentence])[:, :-1]
        tokenized_target_sentence = pad_sequences(tokenized_target_sentence, maxlen=sequence_length - 1, padding='post')
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = tf.argmax(predictions[0, i, :], axis=-1).numpy()
        sampled_token = spa_index_lookup.get(sampled_token_index, '')
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

# Ensure `test_pairs` and `strip_chars` are defined
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    input_sentence = input_sentence.lower()
    input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated = decode_sentence(input_sentence)
    print(f"input: {input_sentence}")
    print(f"translated: {translated}")
    print()


TypeError: list indices must be integers or slices, not tuple

In [35]:
import tensorflow as tf
import numpy as np
import json
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Load the transformer model
transformer = tf.keras.models.load_model('transformer_model')

# Load the tokenization configuration and vocabulary for English and Spanish
with open('eng_vectorization_config.json', 'r', encoding='utf-8') as f:
    eng_vectorization_config = json.load(f)
with open('eng_vocab.json', 'r', encoding='utf-8') as f:
    eng_vocab = json.load(f)

with open('spa_vectorization_config.json', 'r', encoding='utf-8') as f:
    spa_vectorization_config = json.load(f)
with open('spa_vocab.json', 'r', encoding='utf-8') as f:
    spa_vocab = json.load(f)

# Initialize tokenizers
eng_tokenizer = Tokenizer()
eng_tokenizer.word_index = {word: index for index, word in enumerate(eng_vocab)}

spa_tokenizer = Tokenizer()
spa_tokenizer.word_index = {word: index for index, word in enumerate(spa_vocab)}

# Define the function for vectorizing English input sentences
def eng_vectorization(sentences):
    sequences = eng_tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=eng_vectorization_config['max_len'], padding='post')

# Define BeamSearchDecoder class
class BeamSearchDecoder:
    def __init__(self, model, tokenizer, beam_width, max_seq_len):
        self.model = model
        self.tokenizer = tokenizer
        self.beam_width = beam_width
        self.max_seq_len = max_seq_len
        self.start_token = tokenizer.word_index['[start]']
        self.end_token = tokenizer.word_index['[end]']
        self.vocab_size = len(tokenizer.word_index)

    def decode(self, input_seq):
        # Initialize the beam with the start token
        beam = [(input_seq, [self.start_token], 0)]
        
        # Loop until the beam reaches the maximum sequence length
        for _ in range(self.max_seq_len):
            new_beam = []
            for input_seq, output_seq, score in beam:
                # Expand the current output sequence
                output_seq_padded = pad_sequences([output_seq], maxlen=self.max_seq_len, padding='post')
                predictions = self.model.predict([input_seq, output_seq_padded], verbose=0)
                
                # Get the top-k predictions
                top_k_indices = np.argsort(predictions[0, len(output_seq)-1, :])[-self.beam_width:]
                for index in top_k_indices:
                    new_score = score + np.log(predictions[0, len(output_seq)-1, index])
                    new_seq = output_seq + [index]
                    new_beam.append((input_seq, new_seq, new_score))
            
            # Keep the best beam_width sequences
            beam = sorted(new_beam, key=lambda x: x[2], reverse=True)[:self.beam_width]
            
            # If the end token is reached, break
            if all(seq[-1] == self.end_token for _, seq, _ in beam):
                break

        # Return the best sequence
        return beam[0][1]

def sequence_to_text(sequence, tokenizer):
    index_to_word = {index: word for word, index in tokenizer.word_index.items()}
    return ' '.join([index_to_word.get(index, '') for index in sequence if index != 0 and index != tokenizer.word_index.get('[start]', -1) and index != tokenizer.word_index.get('[end]', -1)])

# Translate function using beam search
def translate_sentence(sentence, beam_width=3, max_seq_len=20):
    input_seq = eng_vectorization([sentence])
    beam_search_decoder = BeamSearchDecoder(transformer, spa_tokenizer, beam_width, max_seq_len)
    decoded_sequence = beam_search_decoder.decode(input_seq)
    return sequence_to_text(decoded_sequence, spa_tokenizer)

